<h1>DESCRIPTION</h1>

Help a leading mobile brand understand the voice of the customer by analyzing the reviews of their product on <strong>Amazon</strong> and the topics that customers are talking about. You will perform topic modeling on specific parts of speech. You’ll finally interpret the emerging topics.

<h2>Problem Statement</h2>

A popular mobile phone brand, Lenovo has launched their budget smartphone in the Indian market. The client wants to understand the VOC (voice of the customer) on the product. This will be useful to not just evaluate the current product, but to also get some direction for developing the product pipeline. The client is particularly interested in the different aspects that customers care about. Product reviews by customers on a leading e-commerce site should provide a good view.


<h3>Steps to perform</h3>:

Discover the topics in the reviews and present it to business in a consumable format. Employ techniques in syntactic processing and topic modeling.

Perform specific cleanup, POS tagging, and restricting to relevant POS tags, then, perform topic modeling using LDA. Finally, give business-friendly names to the topics and make a table for business.


<h4>Content</h4>: 

Dataset: ‘K8 Reviews v0.2.csv’

Columns:

Sentiment: The sentiment against the review (4,5 star reviews are positive &#128512;  , 1,2 are negative &#128532;)

Reviews: The main text of the review

<h5>Tasks</h5>: 

<&#10148;>   Read the .csv file using Pandas. Take a look at the top few records.

<&#10148;>   Normalize casings for the review text and extract the text into a list for easier manipulation.

<&#10148;>   Tokenize the reviews using NLTKs word_tokenize function.

<&#10148;>   Perform parts-of-speech tagging on each sentence using the NLTK POS tagger.

<&#10148;>   For the topic model, we should  want to include only nouns.

                 1. Find out all the POS tags that correspond to nouns.

                 2. Limit the data to only terms with these tags.

<&#10148;>  Lemmatize. 

                 1. Different forms of the terms need to be treated as one.

                 2. No need to provide POS tag to lemmatizer for now.

<&#10148;>  Remove stopwords and punctuation (if there are any). 

<&#10148;>  Create a topic model using LDA on the cleaned-up data with 12 topics.

                 1. Print out the top terms for each topic.

                 2. What is the coherence of the model with the c_v metric?

<&#10148;>  Analyze the topics through the business lens.
 
                 1. Determine which of the topics can be combined.

<&#10148;>  Create topic model using LDA with what you think is the optimal number of topics

                 1. What is the coherence of the model?

<&#10148;>  The business should  be able to interpret the topics.

                 1. Name each of the identified topics.

                 2. Create a table with the topic name and the top 10 terms in each to present to the  business.

1.[Read the .csv file using Pandas. Take a look at the top few records.](#Read-the-.csv-file-using-Pandas.-Take-a-look-at-the-top-few-records.)

2.[Normalize casings for the review text](#Normalize-casings-for-the-review-text)

3.[extract the text into a list for easier manipulation](#extract-the-text-into-a-list-for-easier-manipulation)

4.[Tokenize the reviews using NLTKs word_tokenize function](#Tokenize-the-reviews-using-NLTKs-word_tokenize-function)

5.[Perform parts-of-speech tagging on each sentence using the NLTK POS tagger.](#Perform-parts-of-speech-tagging-on-each-sentence-using-the-NLTK-POS-tagger.)

6.[Find out all the POS tags that correspond to nouns.](#Find-out-all-the-POS-tags-that-correspond-to-nouns.)

7.[Lemmatize.](#Lemmatize.)

8.[Print out the top terms for each topic.](#Print-out-the-top-terms-for-each-topic.)

9.[Create topic model using LDA](#Create-topic-model-using-LDA)

10.[What is the coherence of the model?](#What-is-the-coherence-of-the-model?)

11.[Create a table with the topic name and the top 10 terms in each to present to the  business.](#Create-a-table-with-the-topic-name-and-the-top-10-terms-in-each-to-present-to-the-business.)

12.[What is the coherence of the model?](#What-is-the-coherence-of-the-model?)

In [1]:
##Import the Required libraries

import pandas as pd
import numpy as np
import tqdm
import nltk
import os
import texthero as hero
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim

## Read the .csv file using Pandas. Take a look at the top few records.

In [2]:
amazon=pd.read_csv("K8 Reviews v0.2.csv")
amazon.head()

,sentiment,review
0,1,Good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...
3,1,Good
4,0,The worst phone everThey have changed the last...


In [3]:
#check the unique values for sentimment and their counts

amazon['sentiment'].value_counts()

0    7712
1    6963
Name: sentiment, dtype: int64

##  Normalize casings for the review text 

In [4]:
stopnltk=stopwords.words('english')
stop_new=['lenevo']
stopnltk=stopnltk+stop_new


In [5]:
from texthero import preprocessing#create a custom cleaning pipeline
custom_pipeline = [preprocessing.fillna
                   , preprocessing.lowercase
                   , preprocessing.remove_digits
                   , preprocessing.remove_punctuation
                   , preprocessing.remove_diacritics
                   , preprocessing.remove_stopwords
                   , preprocessing.remove_whitespace]
                   #, preprocessing.stem]#pass the custom_pipeline to the pipeline argument
amazon['review'] = hero.clean(amazon['review'], pipeline = custom_pipeline)
amazon.head()


,sentiment,review
0,1,good need updates improvements
1,0,worst mobile bought ever battery draining like...
2,1,get cash back already january
3,1,good
4,0,worst phone everthey changed last phone proble...


## extract the text into a list for easier manipulation

In [6]:
article0=list(amazon.review.values)
article0[0]

'good need updates improvements'

In [7]:
type(article0)

list

## Tokenize the reviews using NLTKs word_tokenize function

In [8]:
article_tkn=[]
for term in article0:
    article_tkn.append(nltk.word_tokenize(term))

article_tkn[0]

['good', 'need', 'updates', 'improvements']

## Perform parts-of-speech tagging on each sentence using the NLTK POS tagger.

In [9]:
article_pos=[]

for term in article_tkn:
    article_pos.append(nltk.pos_tag(term))
    
article_pos[0]

[('good', 'JJ'), ('need', 'NN'), ('updates', 'NNS'), ('improvements', 'NNS')]

## Find out all the POS tags that correspond to nouns.

In [10]:
[term for (term,pos) in article_pos[0] if pos.startswith ('NN')]

['need', 'updates', 'improvements']

In [11]:
onlynouns=[]

for term in article_pos:
    onlynouns.append([term for (term,pos) in term if pos.startswith ('N')])
    
onlynouns[1]

['mobile',
 'hell',
 'backup',
 'hours',
 'uses',
 'lie',
 'amazon',
 'lenove',
 'battery',
 'booster',
 'charger',
 'hours',
 'regret']

## Lemmatize.

In [12]:
lemma=WordNetLemmatizer()
article_lema=[]
for term in onlynouns:
    article_lema.append([lemma.lemmatize(word) for word in term])
    
    
article_lema[0]

['need', 'update', 'improvement']

## Create a topic model using LDA on the cleaned-up data with 12 topics.

In [13]:
dictionary=gensim.corpora.Dictionary(article_lema)

In [14]:
count=0
for k,v in dictionary.iteritems():
    print(k,v)
    count+=1
    if count>12:
        break

0 improvement
1 need
2 update
3 amazon
4 backup
5 battery
6 booster
7 charger
8 hell
9 hour
10 lenove
11 lie
12 mobile


## Print out the top terms for each topic.

In [15]:
bow_corpus=[dictionary.doc2bow(doc) for doc in article_lema]
bow_corpus[0]

[(0, 1), (1, 1), (2, 1)]

In [16]:
document_num=25
bow_doc_x=bow_corpus[document_num]
for i in range(len(bow_doc_x)):
    print("word {} (\"{}\") appears {} time in document# {}".format(bow_doc_x[i][0],dictionary[bow_doc_x[i][0]],bow_doc_x[i][1],
                                                                 document_num))

word 38 ("option") appears 2 time in document# 25
word 50 ("screen") appears 1 time in document# 25
word 71 ("call") appears 1 time in document# 25
word 72 ("cast") appears 1 time in document# 25
word 73 ("hotspot") appears 1 time in document# 25


## Create topic model using LDA 

In [17]:
lda_model=gensim.models.LdaMulticore(bow_corpus,
                                    num_topics=12,
                                    id2word=dictionary,
                                    passes=20,
                                    workers=2)

In [18]:
for idx,topic in lda_model.print_topics():
    print("Topic :{} \n Words :{} ".format(idx,topic))
    print("\n")

Topic :0 
 Words :0.355*"product" + 0.035*"amazon" + 0.030*"return" + 0.016*"experience" + 0.014*"box" + 0.012*"love" + 0.011*"heat" + 0.011*"issue" + 0.011*"buy" + 0.010*"piece" 


Topic :1 
 Words :0.074*"delivery" + 0.045*"super" + 0.040*"awesome" + 0.033*"gallery" + 0.026*"fast" + 0.024*"thanks" + 0.016*"player" + 0.015*"recorder" + 0.015*"music" + 0.015*"facility" 


Topic :2 
 Words :0.107*"network" + 0.051*"sim" + 0.043*"problem" + 0.036*"support" + 0.030*"work" + 0.030*"jio" + 0.029*"issue" + 0.029*"handset" + 0.024*"volta" + 0.019*"time" 


Topic :3 
 Words :0.102*"phone" + 0.047*"issue" + 0.039*"day" + 0.034*"time" + 0.028*"problem" + 0.026*"call" + 0.021*"service" + 0.020*"option" + 0.018*"month" + 0.015*"lenovo" 


Topic :4 
 Words :0.121*"charger" + 0.060*"hai" + 0.052*"turbo" + 0.044*"h" + 0.020*"ho" + 0.015*"hi" + 0.014*"charge" + 0.013*"bhi" + 0.012*"ka" + 0.011*"cable" 


Topic :5 
 Words :0.085*"camera" + 0.042*"phone" + 0.037*"battery" + 0.023*"mode" + 0.014*"perform

## What is the coherence of the model?

In [19]:
from gensim.models import CoherenceModel
def compute_coherence_values(corpus,texts,dictionary,k,a,b):
    lda_model=gensim.models.LdaMulticore(corpus=corpus,
                                        id2word=dictionary,
                                        random_state=100,
                                        chunksize=100,
                                        passes=20,
                                        alpha=a,
                                        eta=b,
                                        per_word_topics=True)
    coherence_model_lda=CoherenceModel(model=lda_model,texts=texts,dictionary=dictionary,coherence='c_v')
    return coherence_model_lda.get_coherence()


In [20]:
grid={}
grid['validation_set']={}
#topics range
min_topics=3
max_topics=19
step_size=3
topics_range=range(min_topics,max_topics,step_size)
#Alpha Parameter
alpha=list(np.arange(0.01,1,0.3))
#Beta Parameter
beta=list(np.arange(0.01,1,0.3))
#Validation Sets
num_of_docs=len(bow_corpus)
corpus_sets=[gensim.utils.ClippedCorpus(bow_corpus,num_of_docs*0.75).corpus,bow_corpus]
corpus_sets[0][0]


[(0, 1), (1, 1), (2, 1)]

In [22]:
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=126)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv =0
                    #print( "i is {}, k is {}, a is {}, b is {}".format(i,k,a,b) )
                    cv = compute_coherence_values(corpus_sets[i],article_lema, dictionary, 
                                        k, a, b)
                    print( "i is {}, k is {}, a is {}, b is {} and final cv is {}".format(i,k,a,b,cv) )
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

  1%|▋                                                                              | 1/126 [01:43<3:36:21, 103.85s/it]

i is 0, k is 3, a is 0.01, b is 0.01 and final cv is 0.3882555592099666


  2%|█▎                                                                             | 2/126 [03:22<3:31:41, 102.43s/it]

i is 0, k is 3, a is 0.01, b is 0.31 and final cv is 0.4380762586310488


  2%|█▉                                                                             | 3/126 [04:59<3:26:20, 100.65s/it]

...

i is 1, k is 18, a is 0.9099999999999999, b is 0.31 and final cv is 0.4798533982144269


191it [6:29:37, 120.30s/it]

i is 1, k is 18, a is 0.9099999999999999, b is 0.61 and final cv is 0.4186159837157392


192it [6:31:14, 122.27s/it]

i is 1, k is 18, a is 0.9099999999999999, b is 0.9099999999999999 and final cv is 0.39262346921900415


In [25]:
tuneResults = pd.read_csv('lda_tuning_results.csv')

In [26]:
tuneResults[tuneResults.Coherence == tuneResults.Coherence.values.ravel().max()]

,Validation_Set,Topics,Alpha,Beta,Coherence
87,75% Corpus,18,0.31,0.91,0.697016


In [27]:
tuneResults[tuneResults.Coherence > 0.58]

,Validation_Set,Topics,Alpha,Beta,Coherence
6,75% Corpus,3,0.31,0.61,0.588532
7,75% Corpus,3,0.31,0.91,0.669951
22,75% Corpus,6,0.31,0.61,0.613788
23,75% Corpus,6,0.31,0.91,0.670873
38,75% Corpus,9,0.31,0.61,0.610504
54,75% Corpus,12,0.31,0.61,0.615580
55,75% Corpus,12,0.31,0.91,0.692610
70,75% Corpus,15,0.31,0.61,0.611006
71,75% Corpus,15,0.31,0.91,0.648530
87,75% Corpus,18,0.31,0.91,0.697016


## Create a table with the topic name and the top 10 terms in each to present to the business.

In [24]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus_sets[0], dictionary)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.204947  0.008395       1        1  17.821960
3      0.094501 -0.048337       2        1  16.159428
10     0.171044 -0.229041       3        1   9.677594
11     0.134027  0.199808       4        1   9.650044
9      0.151459  0.073466       5        1   8.850589
6     -0.129688  0.205281       6        1   7.572256
0     -0.152660  0.008604       7        1   6.729895
7      0.133608  0.047065       8        1   5.864195
2     -0.080824 -0.224795       9        1   5.389410
8     -0.171875  0.013410      10        1   4.709984
4     -0.226499 -0.030519      11        1   4.032820
1     -0.128042 -0.023336      12        1   3.541819, topic_info=          Term         Freq        Total Category  logprob  loglift
17       phone  6709.000000  6709.000000  Default  30.0000  30.0000
58     product  2112.000000  2112.000000  Default  29.0000  29.0000
5      battery  2955.000000  2955.000000  Default  28.0000  28.0000
68      camera  3270.000000  3270.000000  Default  27.0000  27.0000
109    quality  1322.000000  1322.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
69     feature    39.837967   796.145569  Topic12  -4.3651   0.3456
38      option    27.676218   396.377075  Topic12  -4.7294   0.6787
428  complaint    18.746662    45.913155  Topic12  -5.1189   2.4448
53        time    24.232883   978.888855  Topic12  -4.8622  -0.3582
499   earphone    19.762896   131.811584  Topic12  -5.0661   1.4430

[727 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
2545      4  0.926675     Rs
87        5  0.030028      [
87        7  0.930876      [
88        7  0.950904      ]
1549      6  0.939998  a6000
...     ...       ...    ...
352      11  0.969796     ye
35        4  0.119918   year
35        6  0.213187   year
35        8  0.532968   year
35       10  0.119918   year

[1256 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 11, 12, 10, 7, 1, 8, 3, 9, 5, 2])